# [Hate Speech Identification Shared Task](https://multihate.github.io/): Subtask 1A at [BLP Workshop](https://blp-workshop.github.io/) @IJCNLP-AACL 2025

This shared task is designed to identify the type of hate, its severity, and the targeted group from social media content. The goal is to develop robust systems that advance research in this area.

In this subtask, given a Bangla text collected from YouTube comments, categorize whether it contains abusive, sexism, religious hate, political hate, profane, or none.

### Downloading dataset from github

In [36]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev_test.tsv

--2025-08-18 00:57:14--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


HTTP request sent, awaiting response... 200 OK
Length: 8002036 (7.6M) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1A_train.tsv’

blp25_hatespeech_su 100%[===================>]   7.63M  --.-KB/s    in 0.1s    

2025-08-18 00:57:15 (73.3 MB/s) - ‘blp25_hatespeech_subtask_1A_train.tsv’ saved [8002036/8002036]

--2025-08-18 00:57:15--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


HTTP request sent, awaiting response... 200 OK
Length: 585339 (572K) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1A_dev.tsv’

blp25_hatespeech_su 100%[===================>] 571.62K  --.-KB/s    in 0.03s   

2025-08-18 00:57:15 (19.1 MB/s) - ‘blp25_hatespeech_subtask_1A_dev.tsv’ saved [585339/585339]

--2025-08-18 00:57:15--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


HTTP request sent, awaiting response... 200 OK
Length: 548258 (535K) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1A_dev_test.tsv’

blp25_hatespeech_su 100%[===================>] 535.41K  --.-KB/s    in 0.03s   

2025-08-18 00:57:15 (17.8 MB/s) - ‘blp25_hatespeech_subtask_1A_dev_test.tsv’ saved [548258/548258]



### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
# !pip install --upgrade accelerate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


#### importing required libraries and setting up logger

In [3]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

### Defining the training, validation, and test data

In [4]:
train_file = 'blp25_hatespeech_subtask_1A_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1A_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1A_dev_test.tsv'

### Disable wandb

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Setting up the training parameters

In [6]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./distilBERT_m/",
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no",
    report_to=None
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [7]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

08/18/2025 00:06:04 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0 distributed training: True, 16-bits training: False
08/18/2025 00:06:04 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=Non

#### Defining the Model

In [8]:
model_name = 'distilbert-base-multilingual-cased'

#### setting the random seed

In [9]:
set_seed(training_args.seed)

#### Loading data files

In [10]:
l2id = {'None': 0, 'Religious Hate': 1, 'Sexism': 2, 'Political Hate': 3, 'Profane': 4, 'Abusive': 5}
train_df = pd.read_csv(train_file, sep='\t')
# print(train_df['label'])
train_df['label'] = train_df['label'].map(l2id).fillna(0).astype(int)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id).fillna(0).astype(int)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)

08/18/2025 00:06:14 - INFO - __main__ - loading a local file for train
08/18/2025 00:06:14 - INFO - __main__ - loading a local file for validation
08/18/2025 00:06:14 - INFO - __main__ - loading a local file for test


In [11]:
len(test_df['id'])

2512

##### Extracting number of unique labels

In [12]:
# Labels
label_list = raw_datasets["train"].unique("label")
print(label_list)
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

[0, 5, 4, 1, 3, 2]


### Loading Pretrained Configuration, Tokenizer and Model

In [13]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

[INFO|configuration_utils.py:699] 2025-08-18 00:06:24,483 >> loading configuration file config.json from cache at /users/t/p/tprama/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/config.json
[INFO|configuration_utils.py:771] 2025-08-18 00:06:24,499 >> Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_toke

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

[INFO|configuration_utils.py:699] 2025-08-18 00:06:24,595 >> loading configuration file config.json from cache at /users/t/p/tprama/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/config.json
[INFO|configuration_utils.py:771] 2025-08-18 00:06:24,595 >> Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.49.0",
  "vocab_size": 119547
}



vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2050] 2025-08-18 00:06:25,008 >> loading file vocab.txt from cache at /users/t/p/tprama/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/vocab.txt
[INFO|tokenization_utils_base.py:2050] 2025-08-18 00:06:25,008 >> loading file tokenizer.json from cache at /users/t/p/tprama/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/tokenizer.json
[INFO|tokenization_utils_base.py:2050] 2025-08-18 00:06:25,008 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2050] 2025-08-18 00:06:25,009 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2050] 2025-08-18 00:06:25,009 >> loading file tokenizer_config.json from cache at /users/t/p/tprama/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/tokeni

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

[INFO|modeling_utils.py:3982] 2025-08-18 00:06:31,589 >> loading weights file model.safetensors from cache at /users/t/p/tprama/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/45c032ab32cc946ad88a166f7cb282f58c753c2e/model.safetensors
[INFO|modeling_utils.py:4960] 2025-08-18 00:06:31,673 >> Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (

#### Preprocessing the raw_datasets

In [14]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/35522 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

#### Finalize the training data for training the model

In [15]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [16]:
train_dataset

Dataset({
    features: ['id', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 35522
})

#### Finalize the development/evaluation data for evaluating the model

In [17]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

#### Finalize the test data for predicting the unseen test data using the model

In [18]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

#### Log a few random samples from the training set

In [19]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

08/18/2025 00:07:06 - INFO - __main__ - Sample 7296 of the training set: {'id': 660, 'text': 'সরকারের দায়িত্বপ্রাপ্ত সংস্থা সকল বাণিজ্যিক ভবন গুলোকে বছরে অন্তত একবার পরিদর্শন করে রিপোর্ট প্রদান করবেন একটা বাণিজ্যিক ভবনে নিরাপত্তার সকল ব্যবস্থা থাকা উচিত কিন্তু এই সব সম্ভবের দেশে কিছুই করা হয় না এই দেশে মানুষের জীবনের কোন দাম নেই তাই কেউ কোনকিছুর তদারকিরও দরকার মনে করে না', 'label': 3, 'input_ids': [101, 62853, 11421, 100, 978, 22756, 102069, 87664, 17660, 20513, 80198, 15215, 15691, 971, 75762, 950, 30277, 16431, 18243, 32465, 11199, 937, 70115, 13542, 28777, 36213, 29740, 12235, 17511, 11128, 111240, 40433, 13334, 974, 12235, 22335, 47719, 33072, 76677, 948, 11128, 49178, 11737, 28777, 36715, 17660, 20513, 80198, 15215, 15691, 971, 75762, 11199, 967, 27268, 96397, 13542, 72659, 11128, 87664, 970, 15215, 106676, 964, 81661, 66449, 17460, 35003, 14112, 100024, 978, 15002, 111240, 61596, 100047, 88324, 11199, 37855, 14998, 14704, 100, 26109, 14112, 88324, 11199, 109216, 955, 13100, 75762

#### Get the metric function `accuracy`

In [20]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [21]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


#### Data Collator

In [22]:
data_collator = default_data_collator

#### Initialize our Trainer

In [23]:
train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_2477861/1049306949.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


#### Training our model

In [25]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[INFO|trainer.py:2405] 2025-08-18 00:07:32,975 >> ***** Running training *****
[INFO|trainer.py:2406] 2025-08-18 00:07:32,976 >>   Num examples = 35,522
[INFO|trainer.py:2407] 2025-08-18 00:07:32,976 >>   Num Epochs = 1
[INFO|trainer.py:2408] 2025-08-18 00:07:32,976 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:2411] 2025-08-18 00:07:32,976 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:2412] 2025-08-18 00:07:32,977 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2413] 2025-08-18 00:07:32,977 >>   Total optimization steps = 2,221
[INFO|trainer.py:2414] 2025-08-18 00:07:32,978 >>   Number of trainable parameters = 135,329,286


Step,Training Loss
500,1.068400
1000,0.914300
1500,0.867300
2000,0.846300


[INFO|trainer.py:2657] 2025-08-18 00:29:25,159 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)




#### Saving the tokenizer too for easy upload

In [27]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:3942] 2025-08-18 00:30:01,045 >> Saving model checkpoint to ./distilBERT_m/
[INFO|configuration_utils.py:423] 2025-08-18 00:30:01,047 >> Configuration saved in ./distilBERT_m/config.json
[INFO|modeling_utils.py:3040] 2025-08-18 00:30:01,207 >> Model weights saved in ./distilBERT_m/model.safetensors
[INFO|tokenization_utils_base.py:2500] 2025-08-18 00:30:01,209 >> tokenizer config file saved in ./distilBERT_m/tokenizer_config.json
[INFO|tokenization_utils_base.py:2509] 2025-08-18 00:30:01,209 >> Special tokens file saved in ./distilBERT_m/special_tokens_map.json


***** train metrics *****
  epoch                    =        1.0
  total_flos               =  1095664GF
  train_loss               =     0.9131
  train_runtime            = 0:21:52.18
  train_samples            =      35522
  train_samples_per_second =     27.071
  train_steps_per_second   =      1.693


#### Evaluating our model on validation/development data

In [28]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

08/18/2025 00:30:03 - INFO - __main__ - *** Evaluate ***


[INFO|trainer.py:4258] 2025-08-18 00:30:03,397 >> 
***** Running Evaluation *****
[INFO|trainer.py:4260] 2025-08-18 00:30:03,397 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-18 00:30:03,397 >>   Batch size = 16


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.6883
  eval_loss               =     0.7891
  eval_runtime            = 0:00:26.24
  eval_samples            =       2512
  eval_samples_per_second =     95.723
  eval_steps_per_second   =      5.983


### Predecting the test data

In [34]:
model_name = "csebuetnlp/banglabert" 

08/18/2025 00:31:54 - INFO - __main__ - *** Predict ***


[INFO|trainer.py:4258] 2025-08-18 00:31:54,316 >> 
***** Running Prediction *****
[INFO|trainer.py:4260] 2025-08-18 00:31:54,316 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-18 00:31:54,317 >>   Batch size = 16


08/18/2025 00:32:19 - INFO - __main__ - ***** Predict results *****


In [30]:
ids[0]

879187

#### Saving the model into card

In [31]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:449] 2025-08-18 00:31:05,527 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.6882961988449097}]}


In [35]:
!zip subtask_1A.zip ./distilBERT_m/subtask_1A.tsv

updating: distilBERT_m/subtask_1A.tsv (deflated 90%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
import zipfile
import os
zip subtask_1A.zip ./distilBERT_m/subtask_1A.tsv

SyntaxError: invalid syntax (1487713426.py, line 3)